In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url,
)

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

In [13]:
from langchain.tools import tool

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

sql_query.invoke("SELECT * FROM Artist LIMIT 10")

"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [ ]:
from langchain.agents import create_agent

# Not working with granite4, qwen2.5 models, using cloud model instead
model="gpt-5-nano"
agent = create_agent(
    model=model,
    tools=[sql_query]
)

In [28]:
from langchain.messages import HumanMessage

question = HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?")

response = agent.invoke(
    {"messages": [question]}
)

In [29]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content="Who is the most popular artist beginning with 'S' in this database?", additional_kwargs={}, response_metadata={}, id='ce9f2caa-f146-4b6e-b4f5-5fb7250457a1'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1325, 'prompt_tokens': 142, 'total_tokens': 1467, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1280, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CrCQsrHCHqp6VV7xq5R2epSi9GvpG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5d33-6c67-7e81-b243-2992ae4555b9-0', tool_calls=[{'name': 'sql_query', 'args': {'query': "SELECT artist_name FROM artists WHERE LOWER(artist_name) LIKE 's%' ORDER BY popularity DESC LIMIT 1;"}, 'id': 'call_uy

In [31]:
print(response["messages"][-3].tool_calls[0]['args']['query'])

SELECT A.Name AS ArtistName, COUNT(*) AS TrackCount FROM Artist AS A JOIN Album AS AL ON AL.ArtistId = A.ArtistId JOIN Track AS T ON T.AlbumId = AL.AlbumId WHERE LOWER(A.Name) LIKE 's%' GROUP BY A.Name ORDER BY TrackCount DESC LIMIT 1;


In [32]:
print(response["messages"][-1].content)

Smashing Pumpkins.

Reason: Among artists whose names start with 'S', Smashing Pumpkins has the most tracks in this database (34 tracks). If you want a different popularity metric (e.g., total album count, total duration), I can compute that too.
